In [7]:
import numpy as np
APIKEY = '8e92c06c9a91f8d3c00455109e8e609ee4096d62'
USERNAME = 'eschbacher'

def get_auth_client(username=None, api_key=None, cdb_client=None):
    """
    """
    from carto.sql import SQLClient
    from carto.auth import APIKeyAuthClient
    if cdb_client is None:
        BASEURL = 'https://{username}.carto.com/api/'.format(username=username)
        auth_client = APIKeyAuthClient(BASEURL, api_key)
        sql = SQLClient(auth_client)
    elif (username is not None) and (api_key is not None):
        sql = SQLClient(cdb_client)
    else:
        raise Exception("`username` and `api_key` or `cdb_client` has to be "
                        "specified.")
    return sql

In [8]:
# create table
sql = get_auth_client(USERNAME, APIKEY)

In [ ]:
create_table = "CREATE TABLE batch_size_test(rand_num numeric, int_val int, name text);"
resp = sql.send(create_table)
print(resp)

In [18]:
batch_sizes = (100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5500, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000)
names = ['andy', 'corinne', 'mr. pomelo']
query = '''
INSERT INTO batch_size_test(rand_num, int_val, name)
VALUES {values};
'''
value_query = "({rand_num}, {int_val}, '{name}')"

for i in range(25, 27):
    val_entries = []
    batch = int(1000 * i)
    print("Trying batch size {}".format(batch))
    for i in range(batch):
        val_entries.append(value_query.format(rand_num=np.random.random(),
                                              int_val=i,
                                              name=names[i % 3]))
    val_entry = ','.join(val_entries)
    if batch == 26000:
        with open('batch_26k.txt', 'w') as f:
            f.write(val_entry)
    print("num chars in {batch}: {n_chars}".format(batch=batch, n_chars=len(val_entry)))
    resp = sql.send(query.format(values=val_entry))
    print(resp)


Trying batch size 25000
num chars in 25000: 970655
{'rows': [], 'time': 0.092, 'fields': {}, 'total_rows': 25000}
Trying batch size 26000
num chars in 26000: 1009950
{'rows': [], 'time': 0.094, 'fields': {}, 'total_rows': 26000}
